In [1]:
#Importing the important libraries
import pandas as pd
import geopandas as gp
# Folium for plotting maps
import folium
from shapely import Polygon 

In [3]:
df = pd.read_pickle("data/poi/poi_prepared.pickle")

In [8]:
df[df["category"] == "parking"].groupby("hex_9").count()

,amenity,geometry,name,category,latitude,longitude,public_transport,wheelchair,hex_7,hex_8,pol_7,pol_8,pol_9
hex_9,,,,,,,,,,,,,
89f16253423ffff,4,4,0,4,4,4,0,0,4,4,4,4,4
89f16253427ffff,12,12,0,12,12,12,0,0,12,12,12,12,12
89f1625342fffff,13,13,0,13,13,13,0,0,13,13,13,13,13
89f16253433ffff,1,1,0,1,1,1,0,0,1,1,1,1,1
89f16253437ffff,5,5,0,5,5,5,0,0,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89f175a5d4bffff,3,3,0,3,3,3,0,0,3,3,3,3,3
89f175a5d4fffff,6,6,1,6,6,6,0,0,6,6,6,6,6
89f175a5d53ffff,5,5,0,5,5,5,0,0,5,5,5,5,5
